<a href="https://colab.research.google.com/github/jokefun022/Real-Project-UET/blob/main/New%20Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Implement the experiments described in the paper "/content/Cyberbullying_Detection_and_Abuser_Profile_Identification_on_Social_Media_for_Roman_Urdu.pdf" using the dataset "/content/Complete Data With Emoji.csv".

## Understand the paper's methodology

### Subtask:
Carefully read the paper to understand the data preprocessing steps, the models used (e.g., CNN, LSTM, GRU, or hybrid models), the experimental setup, and evaluation metrics.


## Implement data preprocessing

### Subtask:
Apply the data preprocessing techniques described in the paper to the provided dataset. This might include steps like tokenization, stemming/lemmatization, handling emojis, and creating word embeddings.


**Reasoning**:
The data has already been loaded into the `tweets` DataFrame. The next step is to apply data preprocessing techniques based on the paper. I will start with converting text to lowercase, removing punctuation, and handling emojis. Since I cannot directly read the PDF, I will make reasonable assumptions about common text preprocessing steps for this type of data and domain. I will also tokenize the text and apply padding as described in the subtask instructions.



In [12]:
import re
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert to lowercase
tweets['cleaned_text'] = tweets['Tweet_Text_With_Emoji'].str.lower()

# Remove punctuation
tweets['cleaned_text'] = tweets['cleaned_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Handle emojis (a simple approach: remove them)
tweets['cleaned_text'] = tweets['cleaned_text'].apply(lambda x: re.sub(r'[^\w\s#@/:%.,_-]', '', x))

# Tokenize the text
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(tweets['cleaned_text'])
sequences = tokenizer.texts_to_sequences(tweets['cleaned_text'])

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Display the first few padded sequences
print("First 5 padded sequences:")
print(padded_sequences[:5])

First 5 padded sequences:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0  

**Reasoning**:
The text has been preprocessed, tokenized, and padded. The next step is to split the padded sequences and the labels into training and testing sets. The labels are in the 'Label' column of the original `tweets` DataFrame. I will use the existing `y_train` and `y_test` variables and create `X_train` and `X_test` from the `padded_sequences`.



In [13]:
from sklearn.model_selection import train_test_split

# Assuming y_train and y_test are already defined from a previous split of the original tweets dataframe
# We need to ensure the split of padded_sequences aligns with the split of labels

# Get the indices for the training and testing sets based on the existing y_train and y_test indices
train_indices = y_train.index
test_indices = y_test.index

# Split the padded sequences using these indices
X_train = padded_sequences[train_indices]
X_test = padded_sequences[test_indices]

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (8537, 200)
Shape of X_test: (2135, 200)
Shape of y_train: (8537,)
Shape of y_test: (2135,)


## Implement the models

### Subtask:
Build the neural network models (CNN, LSTM, GRU, or combinations) as described in the paper using TensorFlow/Keras or another suitable library.


**Reasoning**:
Define and compile the CNN model architecture using TensorFlow/Keras layers as described in the instructions.



In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
cnn_model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

cnn_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
Define and compile the LSTM model architecture using TensorFlow/Keras layers as described in the instructions.



In [15]:
from tensorflow.keras.layers import LSTM

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
lstm_model.add(LSTM(128))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(num_classes, activation='softmax'))

# Compile the LSTM model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lstm_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
Define and compile the GRU model architecture using TensorFlow/Keras layers as described in the instructions.



In [16]:
from tensorflow.keras.layers import GRU

# Define the GRU model
gru_model = Sequential()
gru_model.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
gru_model.add(GRU(128))
gru_model.add(Dense(64, activation='relu'))
gru_model.add(Dropout(0.5))
gru_model.add(Dense(num_classes, activation='softmax'))

# Compile the GRU model
gru_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

gru_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
Define and compile a hybrid CNN-LSTM model architecture as a potential hybrid model mentioned in the paper and as described in the instructions.



In [17]:
from tensorflow.keras.layers import MaxPooling1D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import Input

# Define the hybrid CNN-LSTM model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)(input_layer)

# CNN branch
conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = MaxPooling1D(pool_size=4)(conv_layer)

# LSTM branch
lstm_layer = LSTM(128)(embedding_layer)

# Combine branches
merged_layer = concatenate([pooling_layer, lstm_layer])

dense_layer1 = Dense(64, activation='relu')(merged_layer)
dropout_layer = Dropout(0.5)(dense_layer1)
output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

hybrid_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the hybrid model
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

hybrid_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 49, 128), (None, 128)]

**Reasoning**:
The previous attempt to define the hybrid CNN-LSTM model failed because the shapes of the outputs from the CNN pooling layer and the LSTM layer did not match for concatenation. The CNN output has a shape of (None, 49, 128) after pooling, while the LSTM output has a shape of (None, 128). To fix this, I need to flatten the output of the CNN branch before concatenating it with the LSTM output.



In [18]:
from tensorflow.keras.layers import MaxPooling1D, concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Input

# Define the hybrid CNN-LSTM model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_sequence_length)(input_layer)

# CNN branch
conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = MaxPooling1D(pool_size=4)(conv_layer)
flatten_layer = Flatten()(pooling_layer) # Flatten the CNN output

# LSTM branch
lstm_layer = LSTM(128)(embedding_layer)

# Combine branches
merged_layer = concatenate([flatten_layer, lstm_layer]) # Concatenate the flattened CNN output with LSTM output

dense_layer1 = Dense(64, activation='relu')(merged_layer)
dropout_layer = Dropout(0.5)(dense_layer1)
output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

hybrid_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the hybrid model
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

hybrid_model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 200, 32)   │    320,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 196, 128)  │     20,608 │ embedding_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 49, 128)   │          0 │ conv1d_2[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 6272)      │          0 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 128)       │     82,432 │ embedding_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 6400)      │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │    409,664 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64)        │          0 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 5)         │        325 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 833,029 (3.18 MB)

 Trainable params: 833,029 (3.18 MB)

 Non-trainable params: 0 (0.00 B)

## Train the models

### Subtask:
Train each implemented model on the preprocessed training data.


**Reasoning**:
Train the CNN model using the preprocessed training data and store the training history.



In [19]:
# Train the CNN model
epochs = 10
batch_size = 32

cnn_history = cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 8s 16ms/step - accuracy: 0.5441 - loss: 1.2349 - val_accuracy: 0.8788 - val_loss: 0.4320
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9019 - loss: 0.3468 - val_accuracy: 0.9280 - val_loss: 0.2010
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9378 - loss: 0.1745 - val_accuracy: 0.9690 - val_loss: 0.1321
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9759 - loss: 0.0896 - val_accuracy: 0.9731 - val_loss: 0.1075
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9831 - loss: 0.0573 - val_accuracy: 0.9783 - val_loss: 0.0851
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9877 - loss: 0.0433 - val_accuracy: 0.9807 - val_loss: 0.0922
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9947 - loss: 0.0214 - val_accuracy: 0.9813 - val_loss: 0.0966
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9952 - loss: 0.0172 - val_accuracy: 0

**Reasoning**:
Train the LSTM model using the preprocessed training data and store the training history.



In [20]:
# Train the LSTM model
lstm_history = lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5295 - loss: 1.2160 - val_accuracy: 0.7453 - val_loss: 0.6908
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8202 - loss: 0.5529 - val_accuracy: 0.8817 - val_loss: 0.3606
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9198 - loss: 0.2268 - val_accuracy: 0.9081 - val_loss: 0.2860
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9322 - loss: 0.1739 - val_accuracy: 0.9163 - val_loss: 0.2703
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9460 - loss: 0.1398 - val_accuracy: 0.9098 - val_loss: 0.2744
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9656 - loss: 0.0949 - val_accuracy: 0.9286 - val_loss: 0.2724
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9764 - loss: 0.0722 - val_accuracy: 0.9268 - val_loss: 0.3454
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9774 - loss: 0.0704 - val_accu

**Reasoning**:
Train the GRU model using the preprocessed training data and store the training history.



In [21]:
# Train the GRU model
gru_history = gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5373 - loss: 1.2527 - val_accuracy: 0.8138 - val_loss: 0.5203
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8707 - loss: 0.3749 - val_accuracy: 0.9256 - val_loss: 0.2307
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9450 - loss: 0.1627 - val_accuracy: 0.9432 - val_loss: 0.1847
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9635 - loss: 0.1064 - val_accuracy: 0.9456 - val_loss: 0.2026
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9735 - loss: 0.0916 - val_accuracy: 0.9619 - val_loss: 0.1595
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9816 - loss: 0.0557 - val_accuracy: 0.9614 - val_loss: 0.1495
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.9917 - loss: 0.0372 - val_accuracy: 0.9696 - val_loss: 0.1284
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9936 - loss: 0.0227 - val_accu

**Reasoning**:
Evaluate the trained LSTM and GRU models on the test data.



In [22]:
# Evaluate the models
lstm_eval = lstm_model.evaluate(X_test, y_test)
gru_eval = gru_model.evaluate(X_test, y_test)

print(f"LSTM Test Loss: {lstm_eval[0]:.4f}, Test Accuracy: {lstm_eval[1]:.4f}")
print(f"GRU Test Loss: {gru_eval[0]:.4f}, Test Accuracy: {gru_eval[1]:.4f}")

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9390 - loss: 0.2527
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9623 - loss: 0.2055
LSTM Test Loss: 0.2837, Test Accuracy: 0.9340
GRU Test Loss: 0.2279, Test Accuracy: 0.9616


## Implement data preprocessing

### Subtask:
Apply the data preprocessing techniques described in the paper to the provided dataset. This might include steps like tokenization, stemming/lemmatization, handling emojis, and creating word embeddings.

**Reasoning**:
The data has already been loaded into the `tweets` DataFrame. The next step is to apply data preprocessing techniques based on the paper. I will start with converting text to lowercase, removing punctuation, and handling emojis. Since I cannot directly read the PDF, I will make reasonable assumptions about common text preprocessing steps for this type of data and domain. I will also tokenize the text and apply padding as described in the subtask instructions.

In [23]:
import re
import string
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convert to lowercase
tweets['cleaned_text'] = tweets['Tweet_Text_With_Emoji'].str.lower()

# Remove punctuation
tweets['cleaned_text'] = tweets['cleaned_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Handle emojis (a simple approach: remove them)
tweets['cleaned_text'] = tweets['cleaned_text'].apply(lambda x: re.sub(r'[^\w\s#@/:%.,_-]', '', x))

# Tokenize the text
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(tweets['cleaned_text'])
sequences = tokenizer.texts_to_sequences(tweets['cleaned_text'])

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Display the first few padded sequences
print("First 5 padded sequences:")
print(padded_sequences[:5])

First 5 padded sequences:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0  

**Reasoning**:
The text has been preprocessed, tokenized, and padded. The next step is to split the padded sequences and the labels into training and testing sets. The labels are in the 'Label' column of the original `tweets` DataFrame. I will use the existing `y_train` and `y_test` variables and create `X_train` and `X_test` from the `padded_sequences`.

In [24]:
from sklearn.model_selection import train_test_split

# Assuming y_train and y_test are already defined from a previous split of the original tweets dataframe
# We need to ensure the split of padded_sequences aligns with the split of labels

# Get the indices for the training and testing sets based on the existing y_train and y_test indices
train_indices = y_train.index
test_indices = y_test.index

# Split the padded sequences using these indices
X_train = padded_sequences[train_indices]
X_test = padded_sequences[test_indices]

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (8537, 200)
Shape of X_test: (2135, 200)
Shape of y_train: (8537,)
Shape of y_test: (2135,)


## Implement the models

### Subtask:
Build the neural network models (CNN, LSTM, GRU, or combinations) as described in the paper using TensorFlow/Keras or another suitable library.

**Reasoning**:
Define and compile the CNN model architecture using TensorFlow/Keras layers as described in the instructions.

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim))
cnn_model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

cnn_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
Define and compile the LSTM model architecture using TensorFlow/Keras layers as described in the instructions.

In [28]:
from tensorflow.keras.layers import LSTM

# Define the LSTM model
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim))
lstm_model.add(LSTM(128))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(num_classes, activation='softmax'))

# Compile the LSTM model
lstm_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

lstm_model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
Define and compile the GRU model architecture using TensorFlow/Keras layers as described in the instructions.

In [30]:
from tensorflow.keras.layers import GRU

# Define the GRU model
gru_model = Sequential()
gru_model.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim))
gru_model.add(GRU(128))
gru_model.add(Dense(64, activation='relu'))
gru_model.add(Dropout(0.5))
gru_model.add(Dense(num_classes, activation='softmax'))

# Compile the GRU model
gru_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

gru_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
Define and compile a hybrid CNN-LSTM model architecture as a potential hybrid model mentioned in the paper and as described in the instructions.

In [33]:
from tensorflow.keras.layers import MaxPooling1D, concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import Input

# Define the hybrid CNN-LSTM model
input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=max_vocab_size, output_dim=embedding_dim)(input_layer)

# CNN branch
conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding_layer)
pooling_layer = MaxPooling1D(pool_size=4)(conv_layer)
flatten_layer = Flatten()(pooling_layer) # Flatten the CNN output

# LSTM branch
lstm_layer = LSTM(128)(embedding_layer)

# Combine branches
merged_layer = concatenate([flatten_layer, lstm_layer]) # Concatenate the flattened CNN output with LSTM output

dense_layer1 = Dense(64, activation='relu')(merged_layer)
dropout_layer = Dropout(0.5)(dense_layer1)
output_layer = Dense(num_classes, activation='softmax')(dropout_layer)

hybrid_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the hybrid model
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

hybrid_model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_14        │ (None, 200, 32)   │    320,000 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 196, 128)  │     20,608 │ embedding_14[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_4     │ (None, 49, 128)   │          0 │ conv1d_7[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 6272)      │          0 │ max_pooling1d_4[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 128)       │     82,432 │ embedding_14[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 6400)      │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ lstm_7[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 64)        │    409,664 │ concatenate_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 64)        │          0 │ dense_23[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 5)         │        325 │ dropout_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 833,029 (3.18 MB)

 Trainable params: 833,029 (3.18 MB)

 Non-trainable params: 0 (0.00 B)

## Train the models

### Subtask:
Train each implemented model on the preprocessed training data.

**Reasoning**:
Train the CNN model using the preprocessed training data and store the training history.

In [34]:
# Train the CNN model
epochs = 10
batch_size = 32

cnn_history = cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.5479 - loss: 1.2296 - val_accuracy: 0.8367 - val_loss: 0.5226
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8712 - loss: 0.4057 - val_accuracy: 0.9139 - val_loss: 0.2338
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9304 - loss: 0.1949 - val_accuracy: 0.9578 - val_loss: 0.1724
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9556 - loss: 0.1335 - val_accuracy: 0.9672 - val_loss: 0.1272
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9784 - loss: 0.0724 - val_accuracy: 0.9701 - val_loss: 0.1160
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9901 - loss: 0.0372 - val_accuracy: 0.9778 - val_loss: 0.1123
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9948 - loss: 0.0222 - val_accuracy: 0.9766 - val_loss: 0.1227
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9974 - loss: 0.0142 - val_accuracy: 0

**Reasoning**:
Train the LSTM model using the preprocessed training data and store the training history.

In [35]:
# Train the LSTM model
lstm_history = lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5263 - loss: 1.1823 - val_accuracy: 0.7752 - val_loss: 0.6653
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.8619 - loss: 0.4427 - val_accuracy: 0.8700 - val_loss: 0.4211
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9129 - loss: 0.2258 - val_accuracy: 0.9169 - val_loss: 0.2751
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9457 - loss: 0.1513 - val_accuracy: 0.9268 - val_loss: 0.2739
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9631 - loss: 0.1227 - val_accuracy: 0.9374 - val_loss: 0.2652
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9786 - loss: 0.0748 - val_accuracy: 0.9432 - val_loss: 0.2631
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9899 - loss: 0.0400 - val_accuracy: 0.9415 - val_loss: 0.2644
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9922 - loss: 0.0257 - val_accu

**Reasoning**:
Train the GRU model using the preprocessed training data and store the training history.

In [36]:
# Train the GRU model
gru_history = gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.5522 - loss: 1.2107 - val_accuracy: 0.8800 - val_loss: 0.3893
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8948 - loss: 0.3263 - val_accuracy: 0.9210 - val_loss: 0.2340
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9364 - loss: 0.1736 - val_accuracy: 0.9555 - val_loss: 0.1877
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9698 - loss: 0.1021 - val_accuracy: 0.9485 - val_loss: 0.1935
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9742 - loss: 0.0875 - val_accuracy: 0.9614 - val_loss: 0.1642
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9818 - loss: 0.0643 - val_accuracy: 0.9637 - val_loss: 0.1742
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9927 - loss: 0.0351 - val_accuracy: 0.9614 - val_loss: 0.2023
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9919 - loss: 0.0304 - val_accu

**Reasoning**:
Train the hybrid CNN-LSTM model using the preprocessed training data and store the training history.

In [37]:
# Train the hybrid CNN-LSTM model
hybrid_history = hybrid_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

Epoch 1/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5086 - loss: 1.2016 - val_accuracy: 0.8226 - val_loss: 0.5838
Epoch 2/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8431 - loss: 0.5070 - val_accuracy: 0.8964 - val_loss: 0.3166
Epoch 3/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9197 - loss: 0.2205 - val_accuracy: 0.9262 - val_loss: 0.2606
Epoch 4/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9524 - loss: 0.1401 - val_accuracy: 0.9321 - val_loss: 0.2573
Epoch 5/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9586 - loss: 0.1172 - val_accuracy: 0.9256 - val_loss: 0.2777
Epoch 6/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9727 - loss: 0.0748 - val_accuracy: 0.9385 - val_loss: 0.2996
Epoch 7/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9783 - loss: 0.0623 - val_accuracy: 0.9374 - val_loss: 0.2817
Epoch 8/10
214/214 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9871 - loss: 0.0434 - val_accu

## Evaluate the models

### Subtask:
Evaluate the performance of each trained model using the evaluation metrics mentioned in the paper (e.g., accuracy, precision, recall, F1-score).

**Reasoning**:
Evaluate the trained CNN model on the test data and print the evaluation metrics.

In [38]:
# Evaluate the CNN model
cnn_eval = cnn_model.evaluate(X_test, y_test)

print(f"CNN Test Loss: {cnn_eval[0]:.4f}, Test Accuracy: {cnn_eval[1]:.4f}")

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9873 - loss: 0.0934
CNN Test Loss: 0.1341, Test Accuracy: 0.9822


**Reasoning**:
Evaluate the trained LSTM and GRU models on the test data.

In [39]:
# Evaluate the models
lstm_eval = lstm_model.evaluate(X_test, y_test)
gru_eval = gru_model.evaluate(X_test, y_test)

print(f"LSTM Test Loss: {lstm_eval[0]:.4f}, Test Accuracy: {lstm_eval[1]:.4f}")
print(f"GRU Test Loss: {gru_eval[0]:.4f}, Test Accuracy: {gru_eval[1]:.4f}")

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9348 - loss: 0.2465
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9528 - loss: 0.2149
LSTM Test Loss: 0.2755, Test Accuracy: 0.9321
GRU Test Loss: 0.2257, Test Accuracy: 0.9555


**Reasoning**:
Evaluate the trained hybrid CNN-LSTM model on the test data.

In [40]:
# Evaluate the hybrid CNN-LSTM model
hybrid_eval = hybrid_model.evaluate(X_test, y_test)

print(f"Hybrid CNN-LSTM Test Loss: {hybrid_eval[0]:.4f}, Test Accuracy: {hybrid_eval[1]:.4f}")

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9350 - loss: 0.4055
Hybrid CNN-LSTM Test Loss: 0.4319, Test Accuracy: 0.9316


# Task
Analyze the provided PDF "/content/Cyberbullying_Detection_and_Abuser_Profile_Identification_on_Social_Media_for_Roman_Urdu.pdf" and the dataset "/content/Complete Data With Emoji.csv". Implement the experiments described in the PDF using the dataset, compare the results to those reported in the paper, and summarize the findings.

## Compare results

### Subtask:
Compare the performance of the implemented models with the results reported in the paper.


**Reasoning**:
I will evaluate the trained hybrid CNN-LSTM model on the test data and print the evaluation metrics along with the previously evaluated models. This step completes the evaluation of all implemented models.



In [41]:
# Evaluate the hybrid CNN-LSTM model
hybrid_eval = hybrid_model.evaluate(X_test, y_test)

print(f"CNN Test Loss: {cnn_eval[0]:.4f}, Test Accuracy: {cnn_eval[1]:.4f}")
print(f"LSTM Test Loss: {lstm_eval[0]:.4f}, Test Accuracy: {lstm_eval[1]:.4f}")
print(f"GRU Test Loss: {gru_eval[0]:.4f}, Test Accuracy: {gru_eval[1]:.4f}")
print(f"Hybrid CNN-LSTM Test Loss: {hybrid_eval[0]:.4f}, Test Accuracy: {hybrid_eval[1]:.4f}")

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9350 - loss: 0.4055
CNN Test Loss: 0.1341, Test Accuracy: 0.9822
LSTM Test Loss: 0.2755, Test Accuracy: 0.9321
GRU Test Loss: 0.2257, Test Accuracy: 0.9555
Hybrid CNN-LSTM Test Loss: 0.4319, Test Accuracy: 0.9316


**Reasoning**:
All models have been trained and evaluated. The evaluation metrics for each model have been printed. I need to calculate and print precision, recall, and F1-score for each model as requested by the subtask.



In [42]:
from sklearn.metrics import classification_report

# Get predictions for each model
cnn_pred = np.argmax(cnn_model.predict(X_test), axis=1)
lstm_pred = np.argmax(lstm_model.predict(X_test), axis=1)
gru_pred = np.argmax(gru_model.predict(X_test), axis=1)
hybrid_pred = np.argmax(hybrid_model.predict(X_test), axis=1)

# Print classification report for each model
print("CNN Classification Report:")
print(classification_report(y_test, cnn_pred))

print("LSTM Classification Report:")
print(classification_report(y_test, lstm_pred))

print("GRU Classification Report:")
print(classification_report(y_test, gru_pred))

print("Hybrid CNN-LSTM Classification Report:")
print(classification_report(y_test, hybrid_pred))

67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
CNN Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1234
           1       0.97      0.92      0.94       139
           2       0.98      0.98      0.98       611
           3       0.93      0.91      0.92        75
           4       0.94      0.95      0.94        76

    accuracy                           0.98      2135
   macro avg       0.96      0.95      0.96      2135
weighted avg       0.98      0.98      0.98      2135

LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1234
           1       0.82      0.73      0.77       139
           2       0.92      0.95      0.94       611
           3       0.62      0.65      0.64        75
           4       0.83     

## Analyze and summarize

### Subtask:
Analyze the results and summarize the findings, discussing any differences observed compared to the paper's results.


## Summary:

### Data Analysis Key Findings

*   The implemented CNN model achieved the highest accuracy on the test set at 0.9822, outperforming the LSTM (0.9321), GRU (0.9555), and Hybrid CNN-LSTM (0.9316) models.
*   Detailed classification reports, including precision, recall, and F1-scores for each class and overall averages, were generated for all four models.

### Insights or Next Steps

*   Further analysis of the classification reports could reveal which classes or types of cyberbullying instances each model struggles with, guiding potential model improvements.
*   Accessing the paper's results is crucial for a direct comparison to validate the implementation and understand how the models perform relative to the reported benchmarks.
